<a href="https://colab.research.google.com/github/simonseo/Computer-Science-Capstone-Lifting-Squares-to-a-Half-Space-a-Specific-Case-of-Lifting-Pseudo-disks/blob/master/Gekko_Problem_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
!pip install gekko
from gekko import GEKKO
from random import uniform
from IPython.display import HTML, SVG
from IPython.core.magic import register_cell_magic


## Parse Input

In [0]:
DEBUG = True
# input_str = "" #@param {type:"string"}

input_str = """4 11
1
2
3
4
1 2
1 3
1 2 3
2 3
2 4
3 4
2 3 4"""


# input_str = """2 3
# 1
# 2
# 1 2"""


# input_str = """3 6
# 1
# 2
# 3
# 1 2
# 1 2 3
# 2 3"""

In [92]:
input_list = [[int(s) for s in el.split()] for el in input_str.split('\n')]
if DEBUG:
  print(input_list)
square_count, demand_count = input_list.pop(0) # square_count + demand_count is the number of R^2 variables


[[4, 11], [1], [2], [3], [4], [1, 2], [1, 3], [1, 2, 3], [2, 3], [2, 4], [3, 4], [2, 3, 4]]


In [93]:
# Initialize M
M = [ [-1 for _ in range(square_count)] for _ in range(demand_count) ]

for i, demand in enumerate(input_list):
  for sq_idx in demand:
    M[i][sq_idx-1] = 1

if DEBUG:
  print(M)

[[1, -1, -1, -1], [-1, 1, -1, -1], [-1, -1, 1, -1], [-1, -1, -1, 1], [1, 1, -1, -1], [1, -1, 1, -1], [1, 1, 1, -1], [-1, 1, 1, -1], [-1, 1, -1, 1], [-1, -1, 1, 1], [-1, 1, 1, 1]]


## Create Problem and Solve

In [0]:


## Create variables and demands
m = GEKKO()
# circle_vars = [ [m.Var(), m.Var()] for _ in range(square_count)]
# region_vars = [ [m.Var(), m.Var()] for _ in range(demand_count)]
circle_vars = [ [m.Var(value=uniform(0,5)), m.Var(value=uniform(0,5))] for _ in range(square_count)]
region_vars = [ [m.Var(value=uniform(0,5)), m.Var(value=uniform(0,5))] for _ in range(demand_count)]


for i, demand in enumerate(M):
  x1, y1 = region_vars[i][0], region_vars[i][1]
  x1.lower = 0
  y1.lower = 0
  x1.upper = 5
  y1.upper = 5
  for j, val in enumerate(demand):
    assert val == 1 or val == -1
    x2, y2 = circle_vars[j][0], circle_vars[j][1]
    x2.lower = 0
    y2.lower = 0
    x2.upper = 5
    y2.upper = 5
    m.Equation(val*((x2-x1)**2+(y2-y1)**2)**(1/2)<val*(1-val/10))

m.solve(disp=False)

## Draw Results

In [0]:
ellipse_string = ""
for circle in circle_vars:
  x, y = circle
  x, y = x.value[0], y.value[0]
  ellipse_string += "ellipse({},{},{},{});".format(x*100,y*100,200,200) # xstart ystart width height

point_string = ""
for point in region_vars:
  x, y = point
  x, y = x.value[0], y.value[0]
  point_string += "ellipse({},{},{},{});".format(x*100,y*100,5,5)


In [104]:
circle_vars, "\n", region_vars

([[[1.2001780942], [2.3707081068]],
  [[2.4037984943], [3.0969559021]],
  [[2.4518672986], [1.969310805]],
  [[3.8825790278], [2.3917288846]]],
 '\n',
 [[[0.69769891947], [2.3091767301]],
  [[2.1713363537], [3.7400789301]],
  [[2.940427851], [1.6842426321]],
  [[4.2615867736], [2.186542531]],
  [[1.6430905401], [2.9185861832]],
  [[1.7439372536], [1.9739369797]],
  [[1.9573691711], [2.5318897522]],
  [[2.4719807263], [2.5633181499]],
  [[3.2134843503], [2.8522607088]],
  [[3.2591170217], [2.028341716]],
  [[3.0389496049], [2.5373794752]]])

In [106]:
HTML('''
<script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.6.0/p5.js"></script>
<script>
new p5();

createCanvas(500, 500)
rect(0,0,500,500)
noFill()
{0}
{1}

</script>
'''.format(ellipse_string, point_string)) 

